## P0 Optimizations
#### Modified by Dom DiPasquale, Mustafa Haddara, and Patrick Jakubowski

In [ ]:
import nbimporter
from Util import printOutput, compareOutput, renderDiff
from CGopts import TESTcommonSubexpressionElimination, assembly

## Introduction
The compiler problem is generally solved; that is, the task of translating an input text based on a known grammar and rules describing the desired output has a generally accepted solution. However, due to the vagueness and abstractness of the grammar and rules, the resulting code may be unnecessarily complex or inefficient.

The job of a compiler optimization is to make the resulting code more efficient, either in memory use or cpu time. In project we implement a few optimizations to the `P0` compiler used throughout the course and demonstrate the reductions in memory use or cpu time from the optimized code compared to the code produced by the original compiler.

## Optimizations Implemented
We implemented 4 optimizations. The first two remove unused code to reduce memory overhead, the second two simplify existing code for faster execution. 
1. Removing Unused Procedures. See [`removeUnusedProcedures()`](/notebooks/CGopts.ipynb#remove-unused-procedures) in CGopts.ipynb for more technical details.
1. Removing Unused Variables. See [`deadStoreElimination()`](/notebooks/CGopts.ipynb#dead-store-elimination) in CGopts.ipynb for more technical details.
1. Dead Store Elimination. See [`removeUnusedVariables()`](/notebooks/CGopts.ipynb#remove-unused-variables) in CGopts.ipynb for more technical details.
1. Common Subexpression Elimination. See [`commonSubexpressionElimination()`](/notebooks/CGopts.ipynb#common-subexpression-elimination) in CGopts.ipynb for more technical details.

## Ordering
We conduct our individual optimizations serially, in the order that we believed would eliminate the largest chunks of code first: procedures are generally larger than the effective "lifetime" of a variable, which in turn are larger than the sizes of the subexpressions we evaluate in Optimization #4. Thus, when moving down to a "smaller" chunk, we can be sure that this chunk is used by the surrounding code, rather than evaluating variables when the containing procedure is never used. Using this method, we can gaurantee that each sequential optimization has to iterate over the least amount code that we deem useful.

## Complexity
Initially, we would have liked to have evaluated our project's complexity by measuring the improvements brought by our optimizations. However, our optimizations' effects are heavily dependant on how optimal the initial input code is, which we believed was a significant variable that we could not effectively predict. Therefore we decided to evaulate the complexity of our project based on how many lines of code needed to be changed, as well as any additional lines of code that needed to be written. 

The majority of the modifications to the compiler were made in the code generation phase. We started with a copy of CGmips.ipynb that we renamed to CGopts.ipynb. In this way, when our optimizations have no effect, we produce the same code as the original P0 code generator.

We wrote and modified approximately 350 lines of code. This includes the initial modifications to the structure of the code generator that allowed us to save and analyze the initial input code and the resulting instructions before they were written to the output, as well as each individual optimization methods. In order to test our optimizations beside the original compiler's output, we built a tool in Util.ipynb called [`compareOutput()`](/notebooks/Util.ipynb), approximately 50 lines of code long. Minimal modifications also needed to be made to P0.ipynb, but none that were significant enough to alter our overall approximation. 

For more details on each individual optimization method, please refer to the "Optimizations Implemented" section above for link to the actual implementations of each optimization method and the relative technical details.

### Time Complexity
Since each optimization is run serially, we can sum the time complexities of all optimizations in order to determine the total time complexity of the optimizer. In the following measures, $n$ is the number of instructions of MIPS code generated by the code generator before optimization and $k$ is the largest number of variables available within a scope; ie. the largest number of local variables declared within a procedure plus the number of global variables declared.

1. Removing Unused Procedures has a worst-case time complexity $O(n)$. We iterate through every instruction once to determine which procedures are called.
1. Removing Dead Stores has a worst-case time complexity $O(n^2)$. We iterate once through every instruction, and if that instruction is a $sw$, it iterates through every previously declared address. If all of the previous instructions are $lw$ instructions, then we must check all of the addresses specified. Therefore (worst-case) we iterate through every instruction once for every instruction; therefore $n^2$.
1. Removing Unused Variables has a worst-case time complexity $O(nk)$. For every instruction, we iterate through all available variables in that scope to determine if they are used in that scope.
1. Removing Common Subexpressions has a worst-case time complexity $O(n \times (\log (n))^2)$. 

Therefore, in total our optimizations have a time complexity $O(n + nk)$; the $n$ and $n \times (\log (n))^2$ terms are "absorbed" by the $n^2$ term.

## Examples

### Unused Procedures Elimination

The following example demonstrates unused procedures elimination. An "unused procedure" is when a procedure is defined, but that procedure is never used within the main program's function call graph.

Notice in the example that although we define procedures b, c and e on lines 6, 22 and 42 respectively, we never call these procedures from the main program's function call graph. Notice in the example that procedure b is called within procedure c, but since procedure c is never called from the main program's function call graph, procedure b is also never called from the main program's function call graph. Notice in the example that procedure e is defined in procedure d and procedure d is called from the main program's function call graph, but procedure e is never called from the main program's function call graph.

Future work includes implementing procedure inlining, which would save the overhead of procedure calls. This would eliminate even more defined procedures and as a result, reduce the amount of jal operations.

In [3]:
compareOutput("""program p;
    var a: integer;
    procedure b(i: integer);
        begin
            a := a + i
        end;
    procedure c(i: integer);
        begin
            a := a + i;
            b(1)
        end;
    procedure d(i: integer);
        procedure e(i: integer);
            begin
                a := a + i
            end;
        begin
            a := a + i
        end;
    begin
        a := a + 1;
        d(1)
    end
    """)

   | Orginal Compiler              | Optimized Compiler            
---|-------------------------------|-------------------------------
 1 |         .data                 |         .data                 
 2 | a_:     .space 4              | a_:     .space 4              
 3 |         .text                 |         .text                 
 4 |         .globl b              |         .globl d              
 5 |         .ent b                |         .ent d                
 6 | b:                            | d:                            
 7 |         sw $fp, -8($sp)       |         sw $fp, -8($sp)       
 8 |         sw $ra, -12($sp)      |         sw $ra, -12($sp)      
 9 |         sub $fp, $sp, 4       |         sub $fp, $sp, 4       
10 |         sub $sp, $fp, 8       |         sub $sp, $fp, 8       
11 |         lw $t1, a_            |         lw $t3, a_            
12 |         lw $t8, 0($fp)        |         lw $t7, 0($fp)        
13 |         add $t1, $t1, $t8     |         add

### Dead Store Elimination
The following example demonstrates dead store elimination. A "dead store" is a memory operation that stores a value to a memory location, but that memory location is never read from for the rest of the program. Thus, this store is unused or "dead".

Notice in the example that although we store the value `5` to variable `b` on line 15, we never read that value from that variable. Thus the second `sw` operation to `b` is unneeded.

Future work includes eliminating unused writes to registers, which would save instructions. However, since these writes are typically `add` or `addi` operations, the only savings we will see will be from fewer operations; `add` and `addi` operations are typically very quick instructions, especially compared to `sw` operations, which can take far longer.

In [4]:
compareOutput("""program p;
    var a,b: integer;
    begin
        b := 4;
        a := 10 + b;
        b := 5;
        write(a)
    end
    """)

   | Orginal Compiler              | Optimized Compiler            
---|-------------------------------|-------------------------------
 1 |         .data                 |         .data                 
 2 | b_:     .space 4              | b_:     .space 4              
 3 | a_:     .space 4              | a_:     .space 4              
 4 |         .text                 |         .text                 
 5 |         .globl main           |         .globl main           
 6 |         .ent main             |         .ent main             
 7 | main:                         | main:                         
 8 |         addi $t1, $0, 4       |         addi $t1, $0, 4       
 9 |         sw $t1, b_            |         sw $t1, b_            
10 |         addi $t8, $0, 10      |         addi $t8, $0, 10      
11 |         lw $t4, b_            |         lw $t4, b_            
12 |         add $t8, $t8, $t4     |         add $t8, $t8, $t4     
13 |         sw $t8, a_            |         sw 

Procedures present an interesting complication; since the `$fp` and `$sp` registers are used across procedures, the stores and loads for each of these registers do not happen in the sequence of instructions. We mark these as special case registers, and any `sw` operations writing to these locations are never removed. Additionally, these registers are always used with relative addresses, and those relative addresses are not always consistent throughout the entire use.

For instance, consider the following `P0` code. The `sw` at the beginning of the function block stores the value of `$fp` into the memory location `$sp - 8` (line 6). However, at the end of the function, the same memory location is refered to by the address `$fp - 4` (line 15). If we only considered exact memory locations, we would remove the first `sw` operations at the beginning of the function block, which would lead to incorrect code. 

In [5]:
compareOutput("""program p;
    procedure f(i: integer);
        begin
        write(i)
        end;
    begin
        f(2)
    end
    """)

   | Orginal Compiler              | Optimized Compiler            
---|-------------------------------|-------------------------------
 1 |         .data                 |         .data                 
 2 |         .text                 |         .text                 
 3 |         .globl f              |         .globl f              
 4 |         .ent f                |         .ent f                
 5 | f:                            | f:                            
 6 |         sw $fp, -8($sp)       |         sw $fp, -8($sp)       
 7 |         sw $ra, -12($sp)      |         sw $ra, -12($sp)      
 8 |         sub $fp, $sp, 4       |         sub $fp, $sp, 4       
 9 |         sub $sp, $fp, 8       |         sub $sp, $fp, 8       
10 |         lw $a0, 0($fp)        |         lw $a0, 0($fp)        
11 |         li $v0, 1             |         li $v0, 1             
12 |         syscall               |         syscall               
13 |         add $sp, $fp, 4       |         add

### Unused Variable Elimination
The following example demonstrates unused variable elimination; we detect the variables never used and remove them from the program. Since we perform the dead store elimination before this optimization, we are able to detect that variable `b` is never used. Therefore, we are allowed to remove that memory declaration in addition to the declaration for  variable `c`, which is never used (read or written to) in the program.

In the cases of unused variables within function calls, we are able to reduce the amount of stack space allocated by the function (line 12) and must also be careful to remap the used variables to their new memory addresses, ie. if we only allocate 16 bytes on the stack, we should not refer to the value at `$sp - 20`.

In [6]:
compareOutput("""program p;
    var a,b,c: integer;
    procedure f;
        var m,n,o: integer;
        begin
        o := 1;
        m := 2;
        write(o)
        end;
    begin
        b := 4;
        a := 10;
        b := 5;
        f();
        write(a)
    end
    """)

   | Orginal Compiler              | Optimized Compiler            
---|-------------------------------|-------------------------------
 1 |         .data                 |         .data                 
 2 | c_:     .space 4              | a_:     .space 4              
 3 | b_:     .space 4              |         .text                 
 4 | a_:     .space 4              |         .globl f              
 5 |         .text                 |         .ent f                
 6 |         .globl f              | f:                            
 7 |         .ent f                |         sw $fp, -4($sp)       
 8 | f:                            |         sw $ra, -8($sp)       
 9 |         sw $fp, -4($sp)       |         sub $fp, $sp, 0       
10 |         sw $ra, -8($sp)       |         sub $sp, $fp, 16      
11 |         sub $fp, $sp, 0       |         addi $t1, $0, 1       
12 |         sub $sp, $fp, 20      |         sw $t1, -12($fp)      
13 |         addi $t1, $0, 1       |         add

#### Global Variables and Procedures
Procedures introduce two problems. First and foremost, we must be careful to not remove a global variable which is used exclusively within procedures. Our intial approach consisted of scanning all "blocks", beginning with the main `program` block, looking for uses of variables. However, if a variable is used exclusively within a procedure, we do not detect that it is used, and therefore miss out on this declaration. The solution in this case is to scan the procedures _first_ and then scan the main `program` block.

In the following example, note that although the global variable is never used in the main `program` block, it is used within the procedure `f`, and therefore is not removed. Since all variables are used in this example, the optimizations have no effect on the produced code.

In [7]:
compareOutput("""program p;
    var a: integer;
    procedure f(i: integer);
        begin
        a := i
        end;
    begin
        f(2)
    end
    """)

   | Orginal Compiler              | Optimized Compiler            
---|-------------------------------|-------------------------------
 1 |         .data                 |         .data                 
 2 | a_:     .space 4              | a_:     .space 4              
 3 |         .text                 |         .text                 
 4 |         .globl f              |         .globl f              
 5 |         .ent f                |         .ent f                
 6 | f:                            | f:                            
 7 |         sw $fp, -8($sp)       |         sw $fp, -8($sp)       
 8 |         sw $ra, -12($sp)      |         sw $ra, -12($sp)      
 9 |         sub $fp, $sp, 4       |         sub $fp, $sp, 4       
10 |         sub $sp, $fp, 8       |         sub $sp, $fp, 8       
11 |         lw $t1, 0($fp)        |         lw $t1, 0($fp)        
12 |         sw $t1, a_            |         sw $t1, a_            
13 |         add $sp, $fp, 4       |         add

The second problem is a result of the `P0` language supporting _shadowing_. Shadowing describes the phenomenon where a local variable has the same name as a global variable, and there hides or "shadows" the global variable. In this case, we must detect that the variable being used is actually a local variable, not a global variable.

The following example demonstrates an unused global variable being removed due to being shadowed.

In [8]:
compareOutput("""program p;
    var a: integer;
    procedure f(i: integer);
        var a: integer;
        begin
        a := i + 1;
        write(a)
        end;
    begin
        f(2)
    end
    """)

   | Orginal Compiler              | Optimized Compiler            
---|-------------------------------|-------------------------------
 1 |         .data                 |         .data                 
 2 | a_:     .space 4              |         .text                 
 3 |         .text                 |         .globl f              
 4 |         .globl f              |         .ent f                
 5 |         .ent f                | f:                            
 6 | f:                            |         sw $fp, -8($sp)       
 7 |         sw $fp, -8($sp)       |         sw $ra, -12($sp)      
 8 |         sw $ra, -12($sp)      |         sub $fp, $sp, 4       
 9 |         sub $fp, $sp, 4       |         sub $sp, $fp, 12      
10 |         sub $sp, $fp, 12      |         lw $t1, 0($fp)        
11 |         lw $t1, 0($fp)        |         add $t1, $t1, 1       
12 |         add $t1, $t1, 1       |         sw $t1, -12($fp)      
13 |         sw $t1, -12($fp)      |         lw 

### Common Subexpression Elimination
The following  example demonstrates common subexpression elimination.  Common subexpressions (CSEs) are instances where two high level operations use a common expression, leading to repeated calculation of previously calculated values.  For instance:

```
program p;
    var a,b,c,d,e,temp: integer;
    procedure f;
        begin
        a := 1;
        b := 2;
        c := a + b;
        d := a + b;
        write(c);
        write(d)
        end;
    procedure g;
        begin
        a := 1;
        b := 2;
        c := a + b;
        a := 3;
        d := a + b;
        write(c);
        write(d)
        end;
    procedure h;
        begin
        a := 1;
        b := 2;
        c := a + (b + 5);
        d := a + (b + (c + 10));
        e := a + (b + (c + 20));
        write(d);
        write(e)
        end;
    procedure x;
        begin
        a := 1;
        b := 2;
        temp := a + b;
        c := temp;
        d := temp;
        write(c);
        write(d)
        end;
    begin
        f();
        g();
        h();
        x();
    end
```

The above functions each display a different aspect of CSEs.  f gives a very basic example of what a common subexpression may look like, where a + b is the CSE between c and d. Now, having identified a CSE, we can now use common subexpression elimination.  First, we store the result of a + b into a new register, then replace the instances of a + b with our temporary variable.  x is an example of what a CSE elimination looks like on a high level.

g shows another important aspect of CSEs, that we cannot assume that just because an expression may contain the same variables, it is a safe elimination.  Doing so here would result in an incorrect output of d, as the value of a has changed.  Therefore we must make sure that no changes to any variables in our CSE were changed between instances of it.  Failing to do so will likely lead to incorrect output.

Finally, h is an example that CSEs are not necessarily only 2 variables in size, they may be any size, however the more complex a CSE the more difficult it is to keep track of its individual components and ensure that no unsafe optimization occurs.  This is partially mitigated by making common subexpression elimination only remove 2 term CSEs and repeatedly run until no more code changes occur, so that at each step two variables are collapsed into one, and if any changes were to occur to any component variables of the CSE, an earlier run of the elimination would correctly determine that the CSE is unsafe.  However, this also may cause the compiler to fail to detect a valid CSE due to 3 or more term CSEs being in different orders, due to commutative operations.

Note that our CSE elimination is unsafe with code that is not loop invariant, as well as some with jumps backward (be it from looping or some if/else branches).  An edge case also exists where if a CSE is contained within an assignment to a variable within that CSE, CSEE does not run for that case (for instance, if a := a + b existed in the above code after it's initial assignment), even though it would be safe to do so (but not after).  This is to prevent a wide variety of issues that happen when an unsafe assignment occurs on the boundaries of our ranges.

Below is an example of code that contains 1 CSE 4 times, with a CSE variable modified between the second and third occurance.  However, when we compile it, we find P0 does not generate these CSEs, as it does not reuse registers for reoccuring (and unchanged) variables.

In [10]:
printOutput("""program p;
    var a,b,c,d,e: integer;
    begin
        a := 1;
        b := 2;
        c := a + b;
        d := b + a;
        a := a + b;
        e := c + (a + b);
        e := 4 + (a + b);
        write(c);
        write(d);
        write(e)
    end
    """)

   | Orginal Compiler              
---|-------------------------------
 1 |         .data                 
 2 | e_:     .space 4              
 3 | d_:     .space 4              
 4 | c_:     .space 4              
 5 | b_:     .space 4              
 6 | a_:     .space 4              
 7 |         .text                 
 8 |         .globl main           
 9 |         .ent main             
10 | main:                         
11 |         addi $t1, $0, 1       
12 |         sw $t1, a_            
13 |         addi $t8, $0, 2       
14 |         sw $t8, b_            
15 |         lw $t4, a_            
16 |         lw $t6, b_            
17 |         add $t4, $t4, $t6     
18 |         sw $t4, c_            
19 |         lw $t0, b_            
20 |         lw $t5, a_            
21 |         add $t0, $t0, $t5     
22 |         sw $t0, d_            
23 |         lw $t2, a_            
24 |         lw $t3, b_            
25 |         add $t2, $t2, $t3     
26 |         sw $t2, a_     

While CSEE can easily run on this code, no changes will be made and therefore to demonstrate CSEE functionality we have manually replaced the generated instructions with more efficient code that reuses registers if it can, generating the CSEs that can now be optimized.  Note that while this code is funtionally similar, it is not the same as shown above.  Above the code is a generated stream of prints that describe what the CSEE function is doing, with the intial step through and how it treats each line, to better demonstrate the structure and steps of the optimization.

In [11]:
test_instructions = [[('', '.data', ''), ('', '.globl main', ''), ('', '.ent main', ''), ('main', '', ''),
                       ('', 'addi $t6, $0, 1', ''), ('', 'sw $t6, a_', ''), ('', 'addi $t4, $0, 2', ''),
                       ('', 'sw $t4, b_', ''),
                       ('', 'add $t5, $t4, $t6', ''), ('', 'sw $t5, c_', ''), ('', 'add $t1, $t4, $t6', ''), ('', 'add $t3, $t4, $t6', ''),
                       ('', 'add $t4, $t5, $t1', ''), ('', 'add $t7, $t6, $t4', ''), ('', 'sw $t7, d_', ''), 
                       ('', 'add $t0, $t4, $t6', ''), ('', 'add $t8, $t5, $t0', ''),
                       ('', 'sw $t8, e_', ''), ('', 'lw $a0, c_', ''), ('', 'li $v0, 1', ''), ('', 'syscall', ''),
                       ('', 'lw $a0, d_', ''), ('', 'li $v0, 1', ''), ('', 'syscall', ''), ('', 'lw $a0, e_', ''),
                       ('', 'li $v0, 1', ''), ('', 'syscall', ''), ('', 'li $v0, 10', ''), ('', 'syscall', ''),
                       ('', '.end main', '')]]

startCode = []
for i in test_instructions[0]:
    startCode.append(assembly(i[0], i[1], i[2]))

renderDiff(startCode,TESTcommonSubexpressionElimination(test_instructions))

Free registers: 
{'$t2'}
----------------------------
instruction number: 0
('', '.data', '')
----------------------------
instruction number: 1
('', '.globl main', '')
----------------------------
instruction number: 2
('', '.ent main', '')
----------------------------
instruction number: 3
('main', '', '')
----------------------------
instruction number: 4
('', 'addi $t6, $0, 1', '')
Modified register detected: $t6
----------------------------
instruction number: 5
('', 'sw $t6, a_', '')
sw detected, skipping
----------------------------
instruction number: 6
('', 'addi $t4, $0, 2', '')
Modified register detected: $t4
----------------------------
instruction number: 7
('', 'sw $t4, b_', '')
sw detected, skipping
----------------------------
instruction number: 8
('', 'add $t5, $t4, $t6', '')
Modified register detected: $t5
New subexpression
----------------------------
instruction number: 9
('', 'sw $t5, c_', '')
sw detected, skipping
----------------------------
instruction number: 